# Objective

In this notebook, I provide a brief analysis of the trader data and predict trader buying patterns and the number of shares purchased.

# Imports

In [123]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Data Preparation

In [91]:
data = pd.read_csv('trader_data_df_1.csv')
data.head()

,Unnamed: 0,Transaction Date,Transaction Type,SYMBOL,Quantity,Price,Amount,Investor #
0,0,2020-09-29,Bought,AAPL,4.0,113.360878,453.443512,0
1,1,2020-09-29,Bought,ACAD,4.0,40.410000,161.639999,0
2,2,2020-09-29,Bought,AMD,4.0,81.769997,327.079987,0
3,3,2020-09-29,Bought,AMWL,4.0,28.100000,112.400002,0
4,4,2020-09-29,Bought,EBS,4.0,104.339996,417.359985,0


Unnamed 0 corresponds to the transaction number from a specific investor, so I will rename it.

In [92]:
data.rename(columns={'Unnamed: 0': 'Transaction #'}, inplace=True)
data.head()

,Transaction #,Transaction Date,Transaction Type,SYMBOL,Quantity,Price,Amount,Investor #
0,0,2020-09-29,Bought,AAPL,4.0,113.360878,453.443512,0
1,1,2020-09-29,Bought,ACAD,4.0,40.410000,161.639999,0
2,2,2020-09-29,Bought,AMD,4.0,81.769997,327.079987,0
3,3,2020-09-29,Bought,AMWL,4.0,28.100000,112.400002,0
4,4,2020-09-29,Bought,EBS,4.0,104.339996,417.359985,0


In [93]:
data.dtypes

Transaction #         int64
Transaction Date     object
Transaction Type     object
SYMBOL               object
Quantity            float64
Price               float64
Amount              float64
Investor #            int64
dtype: object

In [94]:
data['Transaction Date'] = pd.to_datetime(data['Transaction Date'])
data.describe()

,Transaction #,Quantity,Price,Amount,Investor #
count,744.000000,744.000000,744.000000,744.000000,744.000000
mean,64.602151,208.175930,80.177096,6289.240716,2.176075
std,46.126986,1364.445410,77.294823,33859.768439,1.723847
min,0.000000,0.400000,9.165000,10.900000,0.000000
25%,26.000000,2.000000,26.889999,115.989692,1.000000
50%,56.000000,4.976640,61.014999,334.396616,2.000000
75%,97.000000,22.024729,85.470001,1229.133914,3.000000
max,184.000000,23367.209953,419.070007,445681.958769,6.000000


In [95]:
np.unique(data['Transaction Type'])

array(['Bought'], dtype=object)

It looks like there is only buying behaviour. In this case, this feature is not useful and we can discard it.

In [96]:
data.drop(columns=['Transaction Type'], inplace=True)
data.head()

,Transaction #,Transaction Date,SYMBOL,Quantity,Price,Amount,Investor #
0,0,2020-09-29,AAPL,4.0,113.360878,453.443512,0
1,1,2020-09-29,ACAD,4.0,40.410000,161.639999,0
2,2,2020-09-29,AMD,4.0,81.769997,327.079987,0
3,3,2020-09-29,AMWL,4.0,28.100000,112.400002,0
4,4,2020-09-29,EBS,4.0,104.339996,417.359985,0


# Exploratory Data Analysis

First, I will take a look at the total buying frequencies of each investor.

In [98]:
for i in range(7):    
    inv = data.loc[data['Investor #'] == i]
    inv_grouped = inv.groupby('Transaction Date').sum()
    fig = go.Figure(go.Scatter(x=inv_grouped.index, y=inv_grouped['Quantity']))
    fig.update_layout(
        title={
            'text': f'Investor {i} Buying Frequency on Date',
            'x': 0.5
        },
        xaxis_title='Date',
        yaxis_title='Buying Frequency'
    )
    fig.show()

From all the plots, we generally tend to observe a lot of buying behaviour around September to November, which slowly decreases until December. As dicussed in the research article by [Odean](https://onlinelibrary.wiley.com/doi/full/10.1111/0022-1082.00072), this lack of buying may be due to tax planning, where investors tend to sell losses near the end of the year. 
We also seem to observe a lot of buying near the end of the timeframe. Another thing to note is that investors tend to make many purchases at once. We often see spikes rather than curves.

# Modelling

To continue, we fit a simple XGBoost model on the data and record it's RMSE as a metric.

In [122]:
for i in range(7):
    inv = data.loc[data['Investor #'] == i]
    inv_grouped = inv.groupby('Transaction Date').sum()
    X = inv_grouped.drop(columns=['Quantity'])
    y = inv_grouped['Quantity']
    n = len(inv_grouped.index)
    test_index = int(0.8 * n)
    X_train = X.iloc[:test_index]
    y_train = y.iloc[:test_index]
    X_test = X.iloc[test_index:]
    y_test = y.iloc[test_index:]

    xgb = XGBRegressor()
    xgb.fit(X_train, y_train)

    prediction = xgb.predict(X_test)
    
    rmse = mean_squared_error(y_test, prediction) ** (1/2)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=X_train.index, y=y_train, name='Training Data'))
    fig.add_trace(go.Scatter(x=X_test.index, y=prediction, name='Model Prediction'))
    fig.add_trace(go.Scatter(x=X_test.index, y=y_test, name='Truth'))
    fig.update_layout(
        title={
            'text': f'Model Predictions of Investor {i} Buying Frequency on Date (RMSE: {round(rmse, 2)})',
            'x': 0.5
        },
        xaxis_title='Date',
        yaxis_title='Buying Frequency'
    )
    fig.show()


# Conclusions

We note that our model's RMSE is quite high. Despite that, it tends to predict when each investor purchases stocks quite well, but not the amount of stocks purchased. 

With a more in-depth analyses, it may be beneficial to consider classical statistical forecasting. This includes a proper decomposition of seasonality and trend. It would also be interesting to fit various auto-regressive moving averages and exponential smoothing models. 

We should also consider using other machine learning approaches. We can use various classification models, such as KNN and random forest to predict when a purchase may occur. Additionally, we could use those model predictions as an input to a regression model to predict the amount of stock purchased as well. 

Provided with enough data, we could also train a neural network to predict both purchase dates and amounts. 